In [1]:
import numpy as np

In [2]:
from pyiron_atomistics import Project

In [3]:
from pyiron_atomistics.atomistics.master.phonopy import phonopy_to_atoms, atoms_to_phonopy

In [4]:
from phono3py import Phono3py

In [5]:
from phonopy.structure.atoms import PhonopyAtoms

In [6]:
scaled_positions = [
    [0.8750000000000000,  0.8750000000000000,  0.8750000000000000],
    [0.8750000000000000,  0.3750000000000000,  0.3750000000000000],
    [0.3750000000000000,  0.8750000000000000,  0.3750000000000000],
    [0.3750000000000000,  0.3750000000000000,  0.8750000000000000],
    [0.1250000000000000,  0.1250000000000000,  0.1250000000000000],
    [0.1250000000000000,  0.6250000000000000,  0.6250000000000000],
    [0.6250000000000000,  0.1250000000000000,  0.6250000000000000],
    [0.6250000000000000,  0.6250000000000000,  0.1250000000000000],
]

In [7]:
cell = [
    [5.4662628900000003,    0.0000000000000000,    0.0000000000000000],
    [0.0000000000000000,    5.4662628900000003,    0.0000000000000000],
    [0.0000000000000000,    0.0000000000000000,    5.4662628900000003],
]

In [8]:
symbols=["Si"] * 8

In [9]:
atoms = PhonopyAtoms(
    symbols=symbols,
    scaled_positions=scaled_positions,
    cell=cell,
)

In [10]:
pr = Project("calc")

In [11]:
pr.remove_jobs(silently=True, recursive=True)

  0%|          | 0/113 [00:00<?, ?it/s]

In [12]:
job_mini = pr.create.job.Lammps("job_mini")

In [13]:
job_mini.structure = phonopy_to_atoms(atoms)

In [14]:
job_mini.potential = '1994--Tersoff-J--Si-C--LAMMPS--ipr1'

In [15]:
job_mini.calc_minimize(pressure=0)

In [16]:
job_mini.run()

The job job_mini was saved and received the ID: job_mini


In [17]:
atoms = atoms_to_phonopy(job_mini.get_final_structure())

/var/folders/ts/9tzhl05d4gbc4ymttnqmrhvh0000gn/T/ipykernel_15673/1819732388.py:1: DeprecationWarning: pyiron_atomistics.lammps.base.get_final_structure is deprecated: use get_structure() instead.
  atoms = atoms_to_phonopy(job_mini.get_final_structure())


In [18]:
phono = Phono3py(
    unitcell=atoms,
    supercell_matrix=[2,2,2],
    primitive_matrix=[[0, 1/2, 1/2], [1/2, 0, 1/2], [1/2, 1/2, 0]],
    phonon_supercell_matrix=[4,4,4],
    # masses=None,
    # band_indices=None,
    # sigmas=None,
    # sigma_cutoff=None,
    # cutoff_frequency=1e-4,
    # frequency_factor_to_THz=VaspToTHz,
    # is_symmetry=True,
    # is_mesh_symmetry=True,
    # use_grg=False,
    # symmetrize_fc3q=None,
    # store_dense_gp_map=True,
    # store_dense_svecs=True,
    # symprec=1e-5,
    # calculator=None,
    # log_level=0,
    # lapack_zheev_uplo=None,
)

In [19]:
phono.mesh_numbers = [11,11,11]

In [20]:
phono.generate_displacements()
phono.generate_fc2_displacements()

In [21]:
phono.dataset

{'natom': 64,
 'first_atoms': [{'number': 0,
   'direction': [1, 0, 0],
   'displacement': array([3.0000000e-02, 1.8369702e-18, 1.8369702e-18]),
   'id': 1,
   'second_atoms': [{'id': 2,
     'number': 0,
     'direction': [1, 1, 0],
     'displacement': array([2.12132034e-02, 2.12132034e-02, 2.59786817e-18]),
     'pair_distance': 0.0},
    {'id': 3,
     'number': 0,
     'direction': [-1, -1, 0],
     'displacement': array([-2.12132034e-02, -2.12132034e-02, -2.59786817e-18]),
     'pair_distance': 0.0},
    {'id': 4,
     'number': 1,
     'direction': [1, 1, 0],
     'displacement': array([2.12132034e-02, 2.12132034e-02, 2.59786817e-18]),
     'pair_distance': 5.432005264118769},
    {'id': 5,
     'number': 1,
     'direction': [-1, -1, 0],
     'displacement': array([-2.12132034e-02, -2.12132034e-02, -2.59786817e-18]),
     'pair_distance': 5.432005264118769},
    {'id': 6,
     'number': 2,
     'direction': [1, 1, 0],
     'displacement': array([2.12132034e-02, 2.12132034e-02, 

In [22]:
phono.phonon_dataset

{'natom': 512,
 'first_atoms': [{'number': 0,
   'displacement': [0.03, 1.8369701987210304e-18, 1.8369701987210304e-18]}]}

In [23]:
phono.supercells_with_displacements

In [24]:
phono.phonon_supercells_with_displacements

In [25]:
len(phono.supercells_with_displacements)

111

In [26]:
len(phono.phonon_supercells_with_displacements)

1

In [27]:
structure_lst = [phonopy_to_atoms(s) for s in phono.supercells_with_displacements]

In [28]:
structure_phono_lst = [phonopy_to_atoms(s) for s in phono.phonon_supercells_with_displacements]

In [29]:
forces_lst = []
for i, s in enumerate(structure_lst):
    job_displace = pr.create.job.Lammps("job_" + str(i))
    job_displace.structure = s
    job_displace.potential = '1994--Tersoff-J--Si-C--LAMMPS--ipr1'
    job_displace.run()
    forces_lst.append(job_displace["output/generic/forces"][-1])

The job job_0 was saved and received the ID: job_0
The job job_1 was saved and received the ID: job_1
The job job_2 was saved and received the ID: job_2
The job job_3 was saved and received the ID: job_3
The job job_4 was saved and received the ID: job_4
The job job_5 was saved and received the ID: job_5
The job job_6 was saved and received the ID: job_6
The job job_7 was saved and received the ID: job_7
The job job_8 was saved and received the ID: job_8
The job job_9 was saved and received the ID: job_9
The job job_10 was saved and received the ID: job_10
The job job_11 was saved and received the ID: job_11
The job job_12 was saved and received the ID: job_12
The job job_13 was saved and received the ID: job_13
The job job_14 was saved and received the ID: job_14
The job job_15 was saved and received the ID: job_15
The job job_16 was saved and received the ID: job_16
The job job_17 was saved and received the ID: job_17
The job job_18 was saved and received the ID: job_18
The job job_1

In [30]:
forces_phono_lst = []
for i, s in enumerate(structure_phono_lst):
    job_displace = pr.create.job.Lammps("job_phono_" + str(i))
    job_displace.structure = s
    job_displace.potential = '1994--Tersoff-J--Si-C--LAMMPS--ipr1'
    job_displace.run()
    forces_phono_lst.append(job_displace["output/generic/forces"][-1])

The job job_phono_0 was saved and received the ID: job_phono_0


In [31]:
np.array(forces_lst).shape

(111, 64, 3)

In [32]:
phono.forces = np.array(forces_lst)

In [33]:
phono.phonon_forces = np.array(forces_phono_lst)

In [34]:
phono.produce_fc2()

In [35]:
phono.produce_fc3()

In [36]:
phono.init_phph_interaction()

In [37]:
phono.run_phonon_solver()

In [38]:
phono.run_thermal_conductivity()

In [39]:
phono.thermal_conductivity.kappa

array([[[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 1.56971012e+05,  1.56971012e+05,  1.56971012e+05,
          2.40142142e-12,  9.64466513e-13, -1.90953720e-11],
        [ 3.10234270e+04,  3.10234270e+04,  3.10234270e+04,
          3.04328315e-13,  1.89343054e-13, -3.64025417e-12],
        [ 1.71578979e+04,  1.71578979e+04,  1.71578979e+04,
          1.11614176e-13,  8.84244189e-14, -2.03492722e-12],
        [ 1.16433937e+04,  1.16433937e+04,  1.16433937e+04,
          6.32410265e-14,  5.40711016e-14, -1.40571816e-12],
        [ 8.01586545e+03,  8.01586545e+03,  8.01586545e+03,
          4.14113426e-14,  3.62495722e-14, -9.68190484e-13],
        [ 5.55729809e+03,  5.55729809e+03,  5.55729809e+03,
          2.83991109e-14,  2.51456808e-14, -6.63697956e-13],
        [ 3.95506472e+03,  3.95506472e+03,  3.95506472e+03,
          2.02080162e-14,  1.80484766e-14, -4.66134837e-13],
        [ 2.91708237e+03,  2.917